In [2]:
import matplotlib.pyplot as pt
import matplotlib.pyplot
import numpy as np
import cv2
import json

from pathlib import Path
import glob
import os
import torch

In [11]:
def show(*args, labels=[], show_axis=False, figsize=(10, 5), save_to: str = None):
    l = len(args)

    if len(labels) < l:
        labels += [None] * (l - len(labels))

    fg, plots = pt.subplots(int(l ** .5), l // int(l ** .5), figsize=figsize)

    try:
        plots = plots.flatten()
    except:
        plots = [plots]

    for m, p, l in zip(args, plots, labels):
        ax: matplotlib.image.AxesImage = p.imshow(m)

        if not show_axis:
            ax.axes.axis('off')

        if l:
            ax.axes.title.set_text(l)

    if save_to is not None:
        pt.savefig(save_to)

def drawbox(img, boxes, show_indices=False):
    img = img.copy()
    zoom = 1

    if show_indices:
        zoom = 3
        img = cv2.resize(img, np.array(img.shape[:2]) * zoom, interpolation=cv2.INTER_CUBIC)

    for i, box in enumerate(boxes):
        box = box.astype(int) * zoom
        cv2.rectangle(img, box[:2], box[2:], (255, 255, 0), 1)

        if show_indices:
            # x = int(box[[0, 2]].mean().item())
            x = int(box[0].item())
            y = int(box[[1, 3]].mean().item())

            cv2.putText(img, str(i), (x, y), cv2.FONT_HERSHEY_TRIPLEX, .3, (255, 255, 255), 1)

    return img

def fillbox(rimg, boxes, scores, dest=False):
    if not dest:
        rimg = rimg.copy()

    for box, score in sorted(zip(boxes, scores), key=lambda _: _[1], reverse=True):
        cv2.fillPoly(rimg, to_points(box.astype(int)).reshape(1, 4, 2), score, 1)

    return rimg

def readimg(path):
    img = cv2.imread(path)
    cv2.cvtColor(img, cv2.COLOR_BGR2RGB, img)

    return img

def readjson(path):
    dt = json.load(open(path))
    return dt

def to_xywh(boxes):
    if boxes.ndim == 2:
        boxes = np.stack([boxes[:, 0], boxes[:, 1], boxes[:, 2] - boxes[:, 0], boxes[:, 3] - boxes[:, 1]], axis=1)
        return boxes

    elif boxes.ndim == 1:
        return np.array([boxes[0], boxes[1], boxes[2] - boxes[0], boxes[3] - boxes[1]])

def to_points(boxes):
    if boxes.ndim == 2:
        boxes = np.stack([boxes[:, 0], boxes[:, 1], boxes[:, 2], boxes[:, 1], boxes[:, 3], boxes[:, 0], boxes[:, 3]], axis=1)
        return boxes

    elif boxes.ndim == 1:
        return np.array([boxes[0], boxes[1], boxes[2], boxes[1], boxes[2], boxes[3], boxes[0], boxes[3]])


In [16]:
data_root_dir = Path("/Users/shreyas/Developer/Research/GroundingDINO/data/satellite")

images_dir = data_root_dir / "images"
masks_dir = data_root_dir / "masks"
boxes_dir = Path("/Users/shreyas/Developer/Research/GroundingDINO/dataset/resbox")

pdata = {}

for f in glob.glob(str(boxes_dir / "*.json")):
    fname = os.path.basename(f)
    iid, _ = os.path.splitext(fname)

    pdata[iid] = [str(images_dir / f"{iid}.tif"), f]

In [17]:
writedir = Path("/Users/shreyas/Developer/Research/GroundingDINO/dataset/vis/tmp3")

for iid, (imgf, boxf) in pdata.items():
    box = np.array(readjson(boxf), dtype=float)

    img = readimg(imgf)
    img = drawbox(img, box)

    cv2.imwrite(writedir / f"{iid}.png", img[..., ::-1])
